In [158]:
import sklearn as sk
import numpy as np
import pandas as pd
import warnings
import time
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn import model_selection
from sklearn import cross_validation
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
warnings.simplefilter('ignore', DeprecationWarning)

class Scorecard():
    def __init__(self,x,y, max_bins=8, minimum_leaf=0.025):        
        #self.regressor=LogisticRegression() #Regression build method
        self.x = x #Input sample
        self.y = y #Targets
        self.vars = []
        self.vars_after_iv_cut = []
        self.var_list_types = {} #Types of variables
        self.var_list_bins = {} #Binning of scorecard variables dictionary
        self.scorecard_view = pd.DataFrame() #Final scorecard representation
        self.iv_table = {} #information value tables for each variable
        self.gini = int #Gini of model 
        self.logit_model = [] #model object for LogisticRegression
        self.max_bins = max_bins #Regularization parameter. Maximum bins used in decision tree
        self.minimum_leaf = minimum_leaf #Regularization parameter. Mininmum size of one leaf
        self.column = ''
        self.iv_table = {} #Dictionary which contains iv table for each variable
        self.x_one_hot = pd.DataFrame() #Input sample in one-hot view
        self.fill_vars_cats()
        
    
    #Learn model on sample
    def fit(self,x,y,iv_treshold):
        self.x = x
        self.y = y
        #fill all values of var_list_bins
        for col in self.x.columns:
            print(col)
            self.binning(mode_forward='binning',mode_output='normal',column_name=col)  
            #Filling IV table on current variable
            df_t = pd.DataFrame(self.binning(mode_forward='forward',mode_output='normal',column_name=col))
            df_t["y"] = self.y
            #df_t = df_t.rename(index=str, columns = {col:"x"})
            df_iv =pd.DataFrame({'count': df_t.groupby(col)['y'].count(), 
                             'bad_rate': df_t.groupby(col)['y'].mean(),
                             'total_goods': df_t.groupby(col)['y'].count() - df_t.groupby(col)['y'].sum(),
                            'total_bads': df_t.groupby(col)['y'].sum() 
                             }).reset_index()
            df_iv["cumm_bads"] = df_iv['total_bads'].cumsum()
            df_iv["cumm_goods"] = df_iv['total_goods'].cumsum()
            df_iv["cumm_total"] = df_iv['count'].cumsum()
            df_iv["per_bad"] = df_iv["total_bads"]/df_iv["cumm_bads"].max()
            df_iv["per_good"] = df_iv["total_goods"]/df_iv["cumm_goods"].max()
            df_iv["woe"] = np.log((df_iv["per_good"])/(df_iv["per_bad"]+0.000000001))
            iv = (df_iv["per_good"] - df_iv["per_bad"])*np.log((df_iv["per_good"])/(df_iv["per_bad"]+0.000000001))
            df_iv["iv"] = iv.sum()       
            self.iv_table[col] = df_iv
            if df_iv["iv"].mean()>=iv_treshold: self.vars_after_iv_cut.append(col)
        #creating sample in one-hot view
        self.x_one_hot = pd.DataFrame(self.x.index.values)
        for col in self.vars_after_iv_cut:
            self.x_one_hot = pd.merge(self.x_one_hot, pd.DataFrame(self.binning(mode_forward='forward',mode_output='one-hot',column_name=col)),left_index=True,right_index=True)
            del self.x_one_hot[self.x_one_hot.columns[0]]
        
        
    #Input - one variable name 
    #Output - optimal binning, builded on decision tree. Maximum number of bins = max_bins
    def split_numeric(self,column_name):  
        x_train_t = np.array(self.x[column_name][self.x[column_name].notnull()]) #Exclude nulls 
        y_train_t = np.array(self.y[self.x[column_name].notnull()])
        x_train_t = x_train_t.reshape(x_train_t.shape[0], 1) #Need for DecisionTreeClassifier
        m_depth = int(np.log2(self.max_bins)) + 1 #Maximum tree depth
        bad_rate = y.mean()
        start = 1
        cv_scores = []
        cv = 3
        for i in range(start,m_depth): #Loop over all tree depth. CV on the each step
            d_tree = tree.DecisionTreeClassifier(criterion='gini', max_depth=i, min_samples_leaf=self.minimum_leaf)
            scores = cross_val_score(d_tree, x_train_t, y_train_t, cv=cv,scoring='roc_auc')   
            cv_scores.append(scores.mean())        
        best = np.argmax(cv_scores) + start #Criterion - maximum GINI on validation set        
        final_tree = tree.DecisionTreeClassifier(criterion='gini', max_depth=best, min_samples_leaf=0.025) #Build final tree
        final_tree.fit(x_train_t, y_train_t)
        #Final tree
        opt_bins = final_tree.tree_.threshold[final_tree.tree_.feature >= 0]        
        opt_bins = np.append(opt_bins,max(x_train_t)+1)#Add right border
        opt_bins = np.append(opt_bins,min(x_train_t)-1)#Add left border
        opt_bins = np.sort(opt_bins)    
        return opt_bins #Return optimal binning
    
    #Split categorial variable. Grouping variable for regularization.
    #Input = column name
    #Output : add to var_list_bins binned variable as dictionary
    def split_categorial(self,column_name):
        #One-hot encoding
        self.x[column_name] = self.x[column_name].fillna('MISSING')
        x_cat = pd.get_dummies(self.x[column_name],prefix = self.x[column_name].name)
        bad_rate = self.y.mean()
        max_bins = max(self.x[column_name].nunique(),20)
        #Classification by decision tree
        m_depth = max_bins+1
        start = 1
        cv_scores = []
        cv = 3
        for i in range(start,m_depth):
            d_tree = tree.DecisionTreeClassifier(criterion='gini', max_depth=i, min_samples_leaf=self.minimum_leaf) 
            scores = cross_val_score(d_tree, x_cat, self.y, cv=cv,scoring='roc_auc') 
            cv_scores.append(scores.mean())
        #    print("Number of bins = ", i,"; GINI = ",2*scores.mean()-1)
        best = np.argmax(cv_scores) + start #Choose maximizing GINI on validation dataset
        #print("Optimal number of bins: ",best, "; GINI = ",2*max(cv_scores)-1)
        final_tree = tree.DecisionTreeClassifier(criterion='gini', max_depth=best, min_samples_leaf=0.025) #Build final tree
        final_tree.fit(x_cat, self.y)

        #Get leafes names
        x_l = final_tree.apply(x_cat)
        tmp = pd.DataFrame(self.x[column_name])
        tmp["LEAF"] = x_l

        #Make dictionary with optimal binning
        d = {}
        for leaf in tmp["LEAF"].unique():
            d[leaf]=str(self.x[column_name][tmp["LEAF"]==leaf].unique())   
        tmp["x_num"] = tmp["LEAF"].apply(lambda x: d.get(x))
        return d
   
    #Define variable category - numeric or categorial
    #Input - column name
    #Output - numeric or cat
    def check_type(self,column_name):
        from pandas.api.types import is_string_dtype
        from pandas.api.types import is_numeric_dtype   
        #delete nulls
        tmp_var = self.x[column_name][self.x[column_name].notnull()]
        #If number of uniques<=4 then type = categorial
        if tmp_var.nunique()<=4: return 'cat'
        elif is_numeric_dtype(tmp_var): return 'numeric'
        else: return 'cat'
    
    #Fill variable var_list_cats
    def fill_vars_cats(self):
        from pandas.api.types import is_string_dtype
        from pandas.api.types import is_numeric_dtype 
        for col in self.x[self.x.columns]:
            if self.check_type(col)=='numeric': self.var_list_types[col]='numeric'
            if self.check_type(col)=='cat': 
                self.var_list_types[col]='cat'
                if (self.x[col].nunique()<=4)&(is_numeric_dtype(self.x[col])): self.x[col] = self.x[col].apply(lambda x: 'cat_'+str(x))
                
    
    #Add leading zeros to names
    def zero_pad(self,x):
        if str(x)=='MISSING': return '000'
        if len(str(x))==3: return str('00'+str(x))[:-2]+': '
        if len(str(x))==4: return str('0'+str(x))[:-2]+': '
        if len(str(x))==5: str(x)[:-2]+': '
    
    #Naming for categories by rank
    def make_dict(x):        
        x_dict = x.groupby(0)["val"].min().fillna(0).sort_values().reset_index().rename(index=str, columns={0: "x"})
        x_dict['rownum'] = x_dict['val'].rank(method='first', na_option='top')
        x_dict['rownum'] = x_dict['rownum'].apply(zero_pad)
        x_dict['x_num'] = x_dict["rownum"].map(str)+x_dict["x"].map(str)
        del x_dict['val']
        del x_dict['rownum']
        return x_dict   
    
    #Binning procedure
    #Return binned sample. Has two modes - one-hot and norma;
    #Inputs 
    #      x - sample
    #      y - targets
    #      max_bins - maximum number of bins
    #      optimal_bins - for mode_output = 'normal' or 'one-hot' using as input for feed forward
    #                     for mode_forward='binning' calculating of optimal bins
    #                         mode_forward='forward' calculating outputs using optimals bins as input 
    #
    
    #Need for feed forward categorial variables
    #Take value from dictionary var_list_bins and answer if current value is in list
    #If yes - return list
    
    
    def forward_cat(self,x):
        for i in self.var_list_bins[self.column].keys():
            if x in self.var_list_bins[self.column][i]:
                return str(self.var_list_bins[self.column][i]) 
    
    def binning(self,mode_output,mode_forward,column_name):
        variable_type = self.var_list_types[column_name]
        if (variable_type=='numeric')&(mode_forward=='forward'):         
            #Вспомогательная переменная, хранящая разбиения по непустым значениям
            x_bin_t = pd.cut(self.x[column_name][self.x[column_name].notnull()],bins=self.var_list_bins[column_name])    
            #Вспомогательная переменная, хранящая one-hot по непустым значениям
            x_bin = pd.get_dummies(x_bin_t,prefix=self.x[column_name].name,drop_first=True)
            #Добавляем колонку с пустыми значениями
            x_bin[self.x[column_name].name+'_ISNULL']=0
            x_null = pd.DataFrame(self.x[column_name][self.x[column_name].isnull()])
            for i in x_bin.columns:
                x_null[i]=0
            x_null[self.x[column_name].name+'_ISNULL']=1
            del x_null[self.x[column_name].name]
            #Если нет NULL то колонку с dummy is null удаляем   
            if len(self.x[column_name][self.x[column_name].isnull()])==0:
                del x_null[self.x[column_name].name+'_ISNULL']
                del x_bin[self.x[column_name].name+'_ISNULL']
            #Вспомогательная переменная, которая хранит узкий и широкий вид, включая пустые значения    
            x_pivot = pd.concat([x_bin_t,pd.DataFrame(self.x[column_name][self.x[column_name].isnull()])]).sort_index(axis=0)        
            del x_pivot[self.x[column_name].name]
            #Заполняем пустые значения MISSING
            x_pivot = x_pivot.fillna('MISSING')
            x_pivot['val'] = self.x[column_name]        
            #Добавляем категориям индекс (создается справочник)           
            x_dict = x_pivot.groupby(0)["val"].min().fillna(0).sort_values().reset_index().rename(index=str, columns={0: "x"})
            x_dict['rownum'] = x_dict['val'].rank(method='first', na_option='top')
            x_dict['rownum'] = x_dict['rownum'].apply(self.zero_pad)
            x_dict[column_name] = x_dict["rownum"].map(str)+x_dict["x"].map(str)
            del x_dict['val']
            del x_dict['rownum']
            x_d =  x_dict   
            x_pivot["rownum"] = x_pivot.index.values
            x_pivot = pd.merge(x_pivot,x_d,left_on=0,right_on="x").sort_values(by='rownum').reset_index()[column_name]
            #Джойним значения со справочником, удаляем исходные        
            if mode_output=='one-hot': return pd.concat([x_bin,x_null]).sort_index(axis=0) #Возвращаем в виде on-hot                            
            if mode_output=='normal': return x_pivot #Возвращаем в "длинном и узком" виде               
        if (variable_type=='cat')&(mode_forward=='forward'): 
            ####################INPUT CODE HERE#####################
            if mode_output=='normal': 
                self.column = column_name
                return self.x[column_name].apply(self.forward_cat)
            if mode_output=='one-hot': 
                self.column = column_name
                return pd.get_dummies(self.x[column_name].apply(self.forward_cat), drop_first=True,prefix=self.x[column_name].name)
        if (variable_type=='numeric')&(mode_forward=='binning'):
            self.var_list_bins[column_name] = self.split_numeric(column_name)
        if (variable_type=='cat')&(mode_forward=='binning'):                
            self.var_list_bins[column_name] = self.split_categorial(column_name)
            #x_bin = self.split_categorial(column_name)          
            #if mode_output=='one-hot': return pd.get_dummies(x_bin,prefix=self.x[column_name].name,drop_first=True)
            #if mode_output=='normal': return pd.DataFrame(x_bin)
    
     


In [3]:
start_time = time.time()
df = pd.read_csv('C:\\users\\booba\\cc_sample.txt', sep=';',index_col=False, decimal=',') 
print(-start_time + time.time()," sec")
df_train, df_test = train_test_split(df,test_size=0.9)
df_train = df_train.reset_index()
df_test = df_test.reset_index()
x_sample_train = df_train.copy()
x_sample_test = df_test.copy()
x_sample_train = x_sample_train.drop(['CONTRACT_SRC_CODE','SCORE_FINAL','BAD_12_FLAG90','index'], axis=1)
x_sample_test = x_sample_test.drop(['CONTRACT_SRC_CODE','SCORE_FINAL','BAD_12_FLAG90','index'], axis=1)
y_train = df_train["BAD_12_FLAG90"][df_train['BAD_12_FLAG90'].notnull()] 
y_test = df_test["BAD_12_FLAG90"][df_test['BAD_12_FLAG90'].notnull()] 
 

13.066200733184814  sec


In [2]:
x = pd.read_csv('C:\\kaggle\\cc_universal_sample.txt',sep=';',index_col=False)
x.reset_index()
x = x[x['FLG_90_6'].notnull()]
y = x['FLG_90_6']
x = x.drop(['FLG_90_6','FLG_90_12','FLG_90_18','APPLICATION_ID','APPLICATION_DATE'],axis=1)


In [159]:
score1 = Scorecard(x,y,max_bins = 8, minimum_leaf=0.001)

In [160]:
score1.fit(x,y, iv_treshold=0.05)

CLIENT_GROUP
PRODUCT_CAT
MAILRU_HIT_FLG
MAILRU_SCORE
MAILRU_OLD_SCORE
MEGAFON_OLD_SCORE
MEGAFON_NEW_SCORE
BKI_HIT
BKI_SCORE_CC_NEW
CC_LONG_TARGET_SCORE
SCORE_EQ
HIT_EQ
FPS_SCORE
SCORE_B2
SCORE_B3


In [161]:
score1.x_one_hot

,"MAILRU_OLD_SCORE_(0.43, 0.492]","MAILRU_OLD_SCORE_(0.492, 0.626]","MAILRU_OLD_SCORE_(0.626, 0.811]","MAILRU_OLD_SCORE_(0.811, 1.972]",MAILRU_OLD_SCORE_ISNULL,"MEGAFON_OLD_SCORE_(0.00529, 0.00756]","MEGAFON_OLD_SCORE_(0.00756, 0.0115]","MEGAFON_OLD_SCORE_(0.0115, 0.0129]","MEGAFON_OLD_SCORE_(0.0129, 0.0144]","MEGAFON_OLD_SCORE_(0.0144, 0.0778]",...,"SCORE_B2_(535.5, 580.5]","SCORE_B2_(580.5, 643.5]","SCORE_B2_(643.5, 694.5]","SCORE_B2_(694.5, 944.0]","SCORE_B3_(117.5, 136.5]","SCORE_B3_(136.5, 145.5]","SCORE_B3_(145.5, 174.5]","SCORE_B3_(174.5, 178.5]","SCORE_B3_(178.5, 201.5]","SCORE_B3_(201.5, 256.0]"
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,0,1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,0,0,0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0
6,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
7,0,1,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,0,0,0
8,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
9,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [116]:
for c in score1.x.columns:
    print(c, score1.iv_table[c]["iv"].mean())

CLIENT_GROUP 0.06894712899756598
PRODUCT_CAT 0.0
MAILRU_HIT_FLG 0.04539521039894759
MAILRU_SCORE 0.794825296245704
MAILRU_OLD_SCORE 0.5603877141488405
MEGAFON_OLD_SCORE 0.4326063277508752
MEGAFON_NEW_SCORE 0.8174630678058318
BKI_HIT 0.1162937519427785
BKI_SCORE_CC_NEW 0.516170107811937
CC_LONG_TARGET_SCORE 0.991345881990709
SCORE_EQ 0.8068247614969851
HIT_EQ 0.009017274471698046
FPS_SCORE 0.48714001380004984
SCORE_B2 0.9371483465930674
SCORE_B3 0.5309492163037581


In [87]:
score1.binning(column_name = 'FPS_SCORE', mode_forward='binning', mode_output='normal')
score1.binning(column_name = 'MAILRU_HIT_FLG', mode_forward='binning', mode_output='normal')
score1.binning(column_name = 'MAILRU_SCORE', mode_forward='binning', mode_output='normal')


In [95]:
pd.DataFrame(score1.binning(column_name = 'MAILRU_HIT_FLG', mode_forward='forward', mode_output='normal'))

,MAILRU_HIT_FLG
0,['cat_1.0']
1,['cat_1.0']
2,['cat_1.0']
3,['cat_1.0']
4,['cat_0.0']
5,['cat_1.0']
6,['cat_1.0']
7,['cat_1.0']
8,['cat_1.0']
9,['cat_1.0']


In [64]:
    c = 'MAILRU_SCORE'
    df_t = pd.DataFrame(score1.binning(mode_forward='forward',mode_output='normal',column_name=c))
    df_t["y"] = y
            #df_t = df_t.rename(index=str, columns = {col:"x"})
    df_iv =pd.DataFrame({'count': df_t.groupby(c)['y'].count(), 
                             'bad_rate': df_t.groupby(c)['y'].mean(),
                             'total_goods': df_t.groupby(c)['y'].count() - df_t.groupby(c)['y'].sum(),
                            'total_bads': df_t.groupby(c)['y'].sum() 
                             }).reset_index()
            

KeyError: 'MAILRU_SCORE'

In [83]:
df_t

,x_num,y
0,"008: (0.533, 1.948]",0.0
1,"008: (0.533, 1.948]",0.0
2,"008: (0.533, 1.948]",0.0
3,"008: (0.533, 1.948]",0.0
4,001: MISSING,0.0
5,"004: (0.224, 0.314]",0.0
6,"008: (0.533, 1.948]",0.0
7,"007: (0.429, 0.533]",0.0
8,"008: (0.533, 1.948]",0.0
9,"008: (0.533, 1.948]",0.0


In [70]:
df_t.groupby(df_t[1].name)['y'].count()

KeyError: 1

In [62]:
df_iv

,MAILRU_HIT_FLG,bad_rate,count,total_bads,total_goods
0,['cat_0.0'],0.020473,10062,206.0,9856.0
1,['cat_1.0'],0.012971,28910,375.0,28535.0


In [21]:
var

{'var1': [0, 1, 2, 3],
 'var2': {2: array(['D', 'C', 'B', 'A1', 'MISSING'], dtype=object),
  3: array(['E'], dtype=object),
  4: array(['B1'], dtype=object)}}